In [526]:
# Objective:
# To get Jira creation and assigned timestamp

In [550]:
#from urllib.parse import urlencode
import pandas as pd
from datetime import datetime, timedelta
from jira import JIRA

In [551]:
email='<< Email ID>>'
api_token=' <<  Token >> '

jira = JIRA('<< jira_URL >>', basic_auth=(email, api_token))
# where jira_url could be like https://<Domain Name>.atlassian.net

In [552]:
def get_assigned_date(jira_no):
    df = pd.DataFrame()
    issue = jira.issue(jira_no, expand='changelog')
    achangelog = issue.changelog
    for history in achangelog.histories:
        for item in history.items:
            d = {
                'key': jira_no,
                'date': history.created,
                'field': item.field,
                'from': getattr(item, 'from'), # because using item.from doesn't wor
                'fromString' : item.fromString,
                'to': item.to,
                'toString': item.toString
            }
            df=df.append(d,ignore_index=True)
    dt1=df[df.sort_values(by=['date'])['field']=='assignee'].head(1).reset_index(drop=True)
    if dt1.empty:
        return pd.to_datetime(history.created) # If there is no assigned, I will retun create timestamp 
    dt=pd.to_datetime(str(dt1['date'][0]))
    return dt

In [553]:
issues = pd.DataFrame(columns=['key','summary','created','assigned_date','diff_sla','status']) # To store the data in df DataFrame
row = pd.DataFrame(columns=['key','summary','created','assigned_date','diff_sla','status']) # To store the data in df DataFrame

In [554]:
#print list of issues under CE project
for singleIssue in jira.search_issues(jql_str='project = CE'):
    row = pd.DataFrame(columns=['key','summary','created','assigned_date','creator']) # To store the data in df DataFrame
    row = pd.DataFrame({'key':[singleIssue.key],'summary':[singleIssue.fields.summary],'created':[singleIssue.fields.created],'creator':[singleIssue.fields.creator],'status':[singleIssue.fields.status]}) # Convert datetime object in DataFrame
    issues=pd.concat([issues, row], sort=False,axis=0,ignore_index=True) # Append row dataframe to df dataframe

In [555]:
for ind in issues.index:
    issues['assigned_date'][ind]=get_assigned_date(str(issues['key'][ind])) # Get the assigned timestamp
    issues['diff_sla'][ind]= issues['assigned_date'][ind] - pd.to_datetime(issues['created'][ind]) # Calculate the diff between create and assigned timestamp

/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel_launcher.py:17: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


In [556]:
#data masking for demo
issues['summary']='XXXXXXXXXX'
issues['creator']='XXXXXXXXXX'
issues['key']='XXXXXXXXXX'

In [557]:
print ("Jiras Not assigned yet")
issues[issues['diff_sla'] < timedelta(minutes=1)]
#For calculating the SLA, I minus the assigned_date from create_date. get_assigned_date function returned create_date if the jira is not assigned yet.
#If the case is not assigned yet, SLA diff should ne 0 ( create_date - get_assigned_date), however minus of both can return some seconds for some reason.
# Therefore, I am removing delta of 1 minute. Not hoping any engineer to ack and reassign the case in less than 1 minute

Jiras Not assigned yet


,key,summary,created,assigned_date,diff_sla,status,creator
4,XXXXXXXXXX,XXXXXXXXXX,2023-02-15T11:41:35.762-0800,2023-02-15 11:41:37.360000-08:00,0 days 00:00:01.598000,Closed,XXXXXXXXXX
6,XXXXXXXXXX,XXXXXXXXXX,2023-02-21T11:47:44.378-0800,2023-02-21 11:47:44.494000-08:00,0 days 00:00:00.116000,Done,XXXXXXXXXX
9,XXXXXXXXXX,XXXXXXXXXX,2022-07-14T15:44:26.517-0700,2022-07-14 15:44:26.616000-07:00,0 days 00:00:00.099000,Done,XXXXXXXXXX
10,XXXXXXXXXX,XXXXXXXXXX,2023-02-24T11:37:46.198-0800,2023-02-24 11:37:53.468000-08:00,0 days 00:00:07.270000,Needs Triaged,XXXXXXXXXX
11,XXXXXXXXXX,XXXXXXXXXX,2023-02-23T09:03:56.762-0800,2023-02-23 09:03:58.215000-08:00,0 days 00:00:01.453000,To Do,XXXXXXXXXX
25,XXXXXXXXXX,XXXXXXXXXX,2023-02-22T09:43:22.090-0800,2023-02-22 09:43:24.628000-08:00,0 days 00:00:02.538000,To Do,XXXXXXXXXX


In [558]:
print ("SLA miss. Jira is not assigned for more than 30 minutes")
issues[issues['diff_sla'] > timedelta(minutes=30)]

SLA miss. Jira is not assigned for more than 30 minutes


,key,summary,created,assigned_date,diff_sla,status,creator
0,XXXXXXXXXX,XXXXXXXXXX,2022-11-15T11:18:52.991-0800,2022-12-13 13:58:11.768000-08:00,28 days 02:39:18.777000,Done,XXXXXXXXXX
1,XXXXXXXXXX,XXXXXXXXXX,2022-11-15T14:37:59.404-0800,2023-02-16 13:26:58.127000-08:00,92 days 22:48:58.723000,Done,XXXXXXXXXX
3,XXXXXXXXXX,XXXXXXXXXX,2022-12-12T13:00:23.826-0800,2022-12-13 13:39:22.188000-08:00,1 days 00:38:58.362000,Done,XXXXXXXXXX
5,XXXXXXXXXX,XXXXXXXXXX,2023-01-31T08:31:43.766-0800,2023-02-14 14:02:47.208000-08:00,14 days 05:31:03.442000,Done,XXXXXXXXXX
7,XXXXXXXXXX,XXXXXXXXXX,2022-06-06T11:18:00.447-0700,2022-06-22 12:42:04.404000-07:00,16 days 01:24:03.957000,Closed,XXXXXXXXXX
13,XXXXXXXXXX,XXXXXXXXXX,2023-02-14T17:46:13.514-0800,2023-02-21 14:04:56.490000-08:00,6 days 20:18:42.976000,In Progress,XXXXXXXXXX
14,XXXXXXXXXX,XXXXXXXXXX,2023-01-17T11:18:53.970-0800,2023-02-16 13:38:55.029000-08:00,30 days 02:20:01.059000,To Do,XXXXXXXXXX
15,XXXXXXXXXX,XXXXXXXXXX,2022-11-15T15:00:16.113-0800,2023-02-17 12:11:59.548000-08:00,93 days 21:11:43.435000,To Do,XXXXXXXXXX
16,XXXXXXXXXX,XXXXXXXXXX,2022-11-15T15:40:35.315-0800,2023-02-16 13:46:56.265000-08:00,92 days 22:06:20.950000,In Progress,XXXXXXXXXX
17,XXXXXXXXXX,XXXXXXXXXX,2023-01-06T14:53:42.558-0800,2023-02-21 13:53:32.022000-08:00,45 days 22:59:49.464000,In Progress,XXXXXXXXXX


In [559]:
print ("SLA met")
issues[(issues['diff_sla'] <= timedelta(minutes=30)) & (issues['diff_sla'] > timedelta(minutes=1))]

SLA met


,key,summary,created,assigned_date,diff_sla,status,creator
2,XXXXXXXXXX,XXXXXXXXXX,2023-02-14T16:32:41.035-0800,2023-02-14 16:45:42.582000-08:00,0 days 00:13:01.547000,Done,XXXXXXXXXX
8,XXXXXXXXXX,XXXXXXXXXX,2022-07-14T15:57:24.988-0700,2022-07-14 16:13:38.291000-07:00,0 days 00:16:13.303000,Done,XXXXXXXXXX
12,XXXXXXXXXX,XXXXXXXXXX,2023-02-21T12:40:36.803-0800,2023-02-21 12:45:16.242000-08:00,0 days 00:04:39.439000,To Do,XXXXXXXXXX
